### Recommendation System

#### Content-based Filtering


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from underthesea import word_tokenize, pos_tag, sent_tokenize
from gensim import corpora, models, similarities
import re
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

##### Cosin Similarity

In [4]:
df = pd.read_csv("data/items.csv")
df.head()

,product_id,product_name,category,sub_category,image,price,rating,Content_wt
0,190,"Áo ba lỗ thun gân ,form body tôn dáng",Thời Trang Nam,Áo Ba Lỗ,https://cf.shopee.vn/file/2c1ca03f5dc42f316fdf...,86250.0,4.9,"Áo lỗ thun gân , form body tôn_dáng Thời_Trang..."
1,191,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp",Thời Trang Nam,Áo Ba Lỗ,https://cf.shopee.vn/file/c7ea4c6574dc79be6b26...,26800.0,4.9,"Áo Ba_Lỗ Nam_Trắng Chất_Cotton Siêu_Mát , Siêu..."
2,192,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...",Thời Trang Nam,Áo Ba Lỗ,https://cf.shopee.vn/file/6f93bcda10efe374f8cc...,39500.0,4.8,"Áo Ba_Lỗ Nam_Tyasuo chất vải co_dãn mát , khôn..."
3,193,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Thời Trang Nam,Áo Ba Lỗ,https://cf.shopee.vn/file/1d7ed5e34bff8bc8b49a...,16500.0,4.8,ÁO BA_LỖ HÀNG_VIỆT_NAM 100 % COTTON Thời_Trang...
4,194,Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...,Thời Trang Nam,Áo Ba Lỗ,NaN,45000.0,4.8,Áo Thun_Nam Thể_Thao Ba_Lỗ Mẫu_Mới Siêu_Đẹp ( ...


In [5]:
STOP_WORD_FILE = 'data/vietnamese-stopwords.txt'

def getStopWords():
    stop_words = []
    with open(STOP_WORD_FILE, 'r', encoding='utf-8') as file:
        stop_words = file.read()

    stop_words = stop_words.split('\n')
    return stop_words
    
stop_words = getStopWords()
stop_words

['a_lô',
 'a_ha',
 'ai',
 'ai_ai',
 'ai_nấy',
 'ai_đó',
 'alô',
 'amen',
 'anh',
 'anh_ấy',
 'ba',
 'ba_ba',
 'ba_bản',
 'ba_cùng',
 'ba_họ',
 'ba_ngày',
 'ba_ngôi',
 'ba_tăng',
 'bao_giờ',
 'bao_lâu',
 'bao_nhiêu',
 'bao_nả',
 'bay_biến',
 'biết',
 'biết_bao',
 'biết_bao_nhiêu',
 'biết_chắc',
 'biết_chừng_nào',
 'biết_mình',
 'biết_mấy',
 'biết_thế',
 'biết_trước',
 'biết_việc',
 'biết_đâu',
 'biết_đâu_chừng',
 'biết_đâu_đấy',
 'biết_được',
 'buổi',
 'buổi_làm',
 'buổi_mới',
 'buổi_ngày',
 'buổi_sớm',
 'bà',
 'bà_ấy',
 'bài',
 'bài_bác',
 'bài_bỏ',
 'bài_cái',
 'bác',
 'bán',
 'bán_cấp',
 'bán_dạ',
 'bán_thế',
 'bây_bẩy',
 'bây_chừ',
 'bây_giờ',
 'bây_nhiêu',
 'bèn',
 'béng',
 'bên',
 'bên_bị',
 'bên_có',
 'bên_cạnh',
 'bông',
 'bước',
 'bước_khỏi',
 'bước_tới',
 'bước_đi',
 'bạn',
 'bản',
 'bản_bộ',
 'bản_riêng',
 'bản_thân',
 'bản_ý',
 'bất_chợt',
 'bất_cứ',
 'bất_giác',
 'bất_kì',
 'bất_kể',
 'bất_kỳ',
 'bất_luận',
 'bất_ngờ',
 'bất_nhược',
 'bất_quá',
 'bất_quá_chỉ',
 'bất_thình_l

In [6]:
from scipy import sparse
import numpy as np
from tqdm import tqdm

def process_tfidf_in_batches(df, column_name, batch_size=512):
    total_docs = len(df)
    
    # Enhanced TF-IDF configuration
    vectorizer = TfidfVectorizer(
        analyzer='word',
        stop_words=stop_words,
        ngram_range=(1, 2),  # Include unigrams and bigrams
        min_df=2,  # Ignore terms appearing in less than 2 documents
        max_df=0.95,  # Ignore terms appearing in more than 95% of docs
        sublinear_tf=True,  # Apply sublinear scaling to term frequencies
        norm='l2',  # Use L2 normalization
        smooth_idf=True,  # Add 1 to document frequencies to prevent division by zero
        use_idf=True,  # Enable IDF weighting
        strip_accents=None,  # Keep Vietnamese accents
        lowercase=True  # Convert to lowercase
    )
    
    # First fit on entire corpus to get complete vocabulary
    print("Fitting vectorizer on full corpus...")
    vectorizer.fit(df[column_name])
    
    # Process in batches
    all_batches = []
    for start_idx in tqdm(range(0, total_docs, batch_size), desc="Processing TF-IDF"):
        end_idx = min(start_idx + batch_size, total_docs)
        batch = df[column_name][start_idx:end_idx]
        
        # Transform using the full vocabulary
        batch_tfidf = vectorizer.transform(batch)
        all_batches.append(batch_tfidf)
    
    # Combine all batches
    tfidf_matrix = sparse.vstack(all_batches)
    print(f"Processed all {total_docs} documents")
    
    # Print some statistics about the vectorization
    print(f"Vocabulary size: {len(vectorizer.vocabulary_)}")
    print(f"Average features per document: {tfidf_matrix.nnz / tfidf_matrix.shape[0]:.2f}")
    
    return tfidf_matrix, vectorizer

# Usage
tfidf_matrix, vectorizer = process_tfidf_in_batches(df, "Content_wt", batch_size=512)

Fitting vectorizer on full corpus...


Processing TF-IDF: 100%|██████████| 97/97 [00:02<00:00, 39.61it/s]

Processed all 49653 documents
Vocabulary size: 221663
Average features per document: 85.17


In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from tqdm import tqdm

def calculate_similarity_batch(tfidf_matrix, batch_size=256):
    n_samples = tfidf_matrix.shape[0]
    cosine_sim = np.zeros((n_samples, n_samples))
    
    # Process in batches
    for i in tqdm(range(0, n_samples, batch_size), desc="Calculating similarities"):
        end = min(i + batch_size, n_samples)
        batch = tfidf_matrix[i:end]
        
        # Calculate similarity between current batch and all documents
        similarities = cosine_similarity(batch, tfidf_matrix)
        cosine_sim[i:end] = similarities
        
    return cosine_sim

# Usage
cosine_sim = calculate_similarity_batch(tfidf_matrix, batch_size=256)

Calculating similarities: 100%|██████████| 194/194 [01:02<00:00,  3.12it/s]


In [8]:
def preprocess_keyword_text(text):
    # Lowercase and strip
    text = text.lower().strip()
    
    # Remove special characters but keep Vietnamese diacritics
    text = re.sub(r'[^\w\s_]', ' ', text)
    
    # Tokenize using underthesea
    tokens = word_tokenize(text, format="text")
    
    # Remove stop words and short tokens
    tokens = [word for word in tokens.split() if not word in stop_words]
    
    # Join tokens back
    return ' '.join(tokens)
def get_similarity_scores(sims, top_k, exclude_idx=None):
    # Convert similarities to list of (index, score) tuples
    sim_scores = list(enumerate(sims))
    
    # Sort by similarity score in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Exclude the query item if specified
    if exclude_idx is not None:
        sim_scores = [s for s in sim_scores if s[0] != exclude_idx]
    
    return sim_scores[:top_k]

def get_recommendations(query, cosine_sim=cosine_sim, nums=5, by_keyword=False, category_filter=None):
    if not by_keyword:
        # Get recommendations by product ID
        idx = df.index[df['product_id'] == query][0]
        category = df.iloc[idx]['category']
        # Get similarity scores for the product
        sim_scores = get_similarity_scores(cosine_sim[idx], nums+1, exclude_idx=idx)
    else:
        # Preprocess query
        query = preprocess_keyword_text(query)
        
        # Transform query
        query_tfidf = vectorizer.transform([query])
        sims = cosine_similarity(query_tfidf, tfidf_matrix)[0]
        
        # Get initial best match for category
        idx = sims.argmax()
        category = df.iloc[idx]['category']
        
        # Get similarity scores for the query
        sim_scores = get_similarity_scores(sims, nums)
    
    # Filter by category if specified
    if category_filter:
        category = category_filter
        
    # Filter products by category
    filtered_scores = [
        (i, score) for i, score in sim_scores
        if df.iloc[i]['category'] == category
    ][:nums]
    
    # Create recommendations DataFrame
    product_indices = [i[0] for i in filtered_scores]
    recommendations = pd.DataFrame({
        'product_name': df['product_name'].iloc[product_indices],
        'category': df['category'].iloc[product_indices],
        'sub_category': df['sub_category'].iloc[product_indices],
        'content': df['Content_wt'].iloc[product_indices],
        'similarity_score': [score[1] for score in filtered_scores]
    })
    
    return recommendations


In [9]:
# By ID
id_search = 19656
print(f"Recommendations based on Product ID {id_search}:")
recommendations_by_id = get_recommendations(id_search, nums=5, by_keyword=False)
recommendations_by_id

Recommendations based on Product ID 19656:


,product_name,category,sub_category,content,similarity_score
2192,Áo may ô Dệt Kim Đông Xuân 154 A0396,Thời Trang Nam,"Áo Hoodie, Áo Len & Áo Nỉ",Áo may_ô Dệt_Kim_Đông_Xuân 154 A0396 Thời_Tran...,0.349554
194,Áo Ba Lỗ Nam CAO CẤP 100% Cotton,Thời Trang Nam,Áo Ba Lỗ,Áo Ba_Lỗ_Nam CAO_CẤP 100 % Cotton Thời_Trang N...,0.335858
126,Áo Ba Lỗ Nam 100% Cotton Mặc nhà Thoáng Mát,Thời Trang Nam,Áo Ba Lỗ,Áo Ba_Lỗ Nam 100 % Cotton Mặc Thoáng_Mát Thời_...,0.334191
661,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Thời Trang Nam,Áo Ba Lỗ,ÁO BA_LỖ HÀNG_VIỆT_NAM 100 % COTTON Thời_Trang...,0.330581
56,[Xả Kho]Áo Ba Lỗ Nam 100% Cotton Mặc nhà Thoán...,Thời Trang Nam,Áo Ba Lỗ,[ Xả Kho ]_Áo Ba_Lỗ Nam 100 % Cotton Mặc Thoán...,0.330022


1. **Độ liên quan về mặt nội dung:**
- 4/5 sản phẩm được gợi ý là áo ba lỗ nam, phù hợp với sản phẩm gốc
- Một sản phẩm thuộc sub-category khác (Áo Hoodie, Áo Len & Áo Nỉ) nhưng vẫn liên quan đến áo
- Hầu hết sản phẩm đều có đặc điểm chung: làm từ cotton, hàng Việt Nam
- Điểm tích cực là mô hình đã nắm bắt được đặc trưng chính về chất liệu và loại sản phẩm

2. **Điểm similarity score:**
- Các scores dao động từ 0.330 đến 0.349
- Mức độ tương đồng này khá tốt (>0.3)
- Điểm số phân bố đều và hợp lý giữa các sản phẩm
- Sản phẩm có điểm cao nhất (0.349) tuy khác sub-category nhưng vẫn liên quan về mặt sản phẩm

In [10]:
# By keyword
keyword_search = "áo thun nam cotton"
print("Recommendations for keyword:", keyword_search)
recommendations_by_keyword = get_recommendations("áo thun nam cotton", nums=5, by_keyword=True)
recommendations_by_keyword

Recommendations for keyword: áo thun nam cotton


,product_name,category,sub_category,content,similarity_score
6499,"Áo thun nam Jordan J2 cá tính ,áo nam vải cott...",Thời Trang Nam,Áo,"Áo thun nam Jordan_J2 cá_tính , áo nam vải cot...",0.292561
21357,Áo thun nam ngắn tay cotton tinh khiết cổ V 20...,Thời Trang Nam,Đồ Hóa Trang,Áo thun nam ngắn cotton tinh_khiết cổ V 2021 p...,0.255124
1095,Áo thun nam ba lỗ in số 79- FORMEN SHOP- FMTD038,Thời Trang Nam,Áo Ba Lỗ,Áo thun nam lỗ in 79 - FORMEN_SHOP - FMTD038 T...,0.250886
6911,Áo phông trơn thun nam cotton 100% cổ tròn tay...,Thời Trang Nam,Áo,Áo phông trơn thun nam cotton 100 % cổ tròn ng...,0.245308
6504,Áo thun nam ngắn tay in hình D A S không cổ Na...,Thời Trang Nam,Áo,Áo thun nam ngắn in hình_D A_S không cổ_Nam_Nữ...,0.232716


1. **Độ liên quan về mặt nội dung:**
- 3/5 sản phẩm thuộc sub-category "Áo", phù hợp với từ khóa tìm kiếm
- Các sản phẩm đều là áo thun nam, đúng với yêu cầu tìm kiếm
- Hầu hết sản phẩm đều đề cập đến chất liệu cotton
- Mô hình đã nắm bắt tốt các từ khóa chính (áo thun, nam, cotton)

2. **Điểm similarity score:**
- Các scores dao động từ 0.232 đến 0.292
- Mức độ tương đồng này hơi thấp (<0.3)
- Điểm số giảm dần hợp lý từ sản phẩm liên quan nhất
- Sản phẩm có điểm cao nhất (0.292) là áo thun nam cotton, hoàn toàn phù hợp với từ khóa